In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
sys.path.insert(0, "../code-previous")
sys.path.insert(0, "..")

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import sklearn.feature_extraction as skfeatures
import utils
import time
import os

import nltk
stopwords = nltk.corpus.stopwords.words('english')

from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set()

import matplotlib.pyplot as plt


path2rawdata = '/mnt/disks/vault/wos2017-parsed/'

In [4]:
raw_data_path = "/mnt/disks/vault/analysis-data/raw_data_full/raw_data_full.pql"
data = pd.read_pickle(raw_data_path)

In [5]:
# New fields to dataframe 
data['log_c5'] = data['c5'].progress_apply(lambda x: np.log(x+1))
data['character_count'] = data['Title'].progress_apply(lambda x: len(x))
data["word_count"] = data["Title"].progress_apply(lambda x: len(x.split()))
data['title_without_punct'] = data['Title'].progress_apply(lambda x: 
                                                           str.lower(''.join(ch for ch in str(x) if category(ch)[0] != 'P')))
data["title_without_stopwords"] = data['title_without_punct'].progress_apply(lambda x: 
                                                                            " ".join([word for word in x.split() if word not in stopwords]))


data["word_count"] = data["Title"].progress_apply(lambda x: len(x.split()))
data["cleaned_title_word_count"] = data["title_without_stopwords"].progress_apply(lambda x: len(x.split()))

Progress: 100%|██████████| 738469/738469 [00:01<00:00, 617868.22it/s]


In [ ]:
grouped_titles = data.groupby('title_without_punct').size().reset_index(name='times_repeated')
#tmp = grouped_titles.sort_values('counts', ascending=False)
select_titles = grouped_titles[grouped_titles.times_repeated >= 2]
#titles = list(select_titles["title_without_punct"])


# filter the data
#_data = data[data.title_without_punct.isin(titles)]

_data = select_titles.merge(data,
                           how = 'left',
                           left_on = 'title_without_punct',
                           right_on = 'title_without_punct')

_data = _data[['Title','PubYear_x', 'Journal', 'c5', 'log_c5', 'times_repeated']]
_data = _data.sort_values(by=['times_repeated','Title'], ascending=False)

_data[:10]

In [ ]:
# lets see if there are articles with the same title - group by article names





In [ ]:
# distance between every article titles
